In [2]:
import pandas as pd
import re, math
import seaborn as sns

from matplotlib import pyplot as plt
from glob import glob
from pprint import pprint

pd.options.display.float_format = "{:,.2f}".format

# Config Session

In [3]:
PATH_OUTPUT = 'data/outputs'

PROTEINS = {
    1: 'CARNES',
    2: 'CARNES ULTRAPROCESSADAS',
    3: 'PESCADOS',
    4: 'OVOS',
    5: 'SUBSTITUTOS DE CARNE'
}

CODES = {
    11: 'CARNE BOVINA',
    12: 'CARNE SUINA',
    13: 'OUTRAS CARNES',
    14: 'AVES',
    15: 'CARNES SECAS E SALGADAS',
    16: 'VÍSCERAS',
    17: 'PREPARAÇÕES QUE INCLUEM CARNES BOVINAS',
    17: 'PREPARAÇÕES QUE INCLUEM OUTRAS CARNES',
    21: 'ALMÔNDEGAS, HAMBURGERS E NUGGETS',
    22: 'EMBUTIDOS, FRIOS E CONSERVAS',
    23: 'PREPARAÇÕES QUE INCLUEM CARNES UP',	
    31: 'PESCADOS FRESCOS',
    32: 'PESCADOS EM CONSERVA',
    33: 'PESCADOS SALGADOS',
    34: 'PREPARAÇÕES QUE INCLUEM PESCADOS',
    41: 'OVOS INTEIROS',
    42: 'REFEIÇÕES QUE INCLUEM OVOS',
    51: 'PTS - CARNE DE SOJA',
    52: 'PREPARAÇÕES QUE INCLUEM CARNE DE SOJA'
}

CORTES = {
    111: 'CORTE TRASEIRO',
    112: 'CORTE DIANTEIRO'
}

PREMISSAS = {
    'ovos': {
        'peso_un_ovo': float(0.05), # peso de um ovo médio 0,05 kg
        'peso_un_ovo_grande': float(0.06), # peso de um ovo grande 0,06 kg
        'BANDEJA': 30, # uma bandeja tem x unidades
        'PLACA': 30,
        'DZ': 12,
        'CARTELA': 15,
        'PEN': 30,
        'CX': [30, 12] # 1 caixa tem 30 dúzias de ovos
    },
    'sardinha':{
        'LATA': 0.125
    }
}


# Input Data

In [4]:
df_dataset_beef = pd.read_parquet('data/banco_carnes_v2.parquet')
df_dataset_beef['produtos'] = df_dataset_beef['codigo'].map(CODES)

# Functions

In [5]:
import re

def standardize_unit(unidade):
    """
    Padroniza as unidades de medida com base em padrões gerais do mercado e abreviações.

    Parâmetro:
        unidade (str): O valor da unidade a ser padronizado.

    Retorna:
        str: Unidade padronizada.
    """
    unidade = str(unidade).strip().upper()

    # Remover asteriscos, vírgulas, e espaços extras
    unidade = re.sub(r'[\*\s,]+', '', unidade)
    #unidade = re.sub(r'[,\*]', '', unidade)
    
    padroes = {
        r'^(KG|KGS?|KGI|KILOGRAMA|KILOS?|KGM|KF|KU|KGR|KGB|KG2\.5|KG2|KG,|KG\'|KGL|KG.*)$': 'KG',
        r'^(GRAMA|GRAMAS|G|GM|GMS|G,)$': 'GRAMA',
        r'^(500G)$': '500G',
        r'^(400G)$': '400G',
        r'^(125G)$': '125G',
        r'^(TON|TONELADA|TNS?|TONS?)$': 'TON',
        r'^(LITRO|LTR|LTS?|LITROS?|LT|LT/84G|LT420G|LT125|LATA1)$': 'LITRO',
        r'^(ML|MILILITRO|MILILITROS)$': 'ML',
        r'^(CX|CAIXA|CAIXAS|CXS?|CXA?|CX18|CXA1|CXA3|CTE)$': 'CX',
        r'^(CX12)$': 'CX10',
        r'^(CX20)$': 'CX20',
        r'^(CX30)$': 'CX30',
        r'^(CX50)$': 'CX50',
        r'^(CX6UND)$': 'CX6',
        r'^(CX60UND|CX60)$': 'CX60',
        r'^(UN|UND|UNIT|UNID|UNIDADES?|UNS?|UID|UND1|30UND|6UND|UNITAR|UN0030|UND.12|6.0UND|UN1|U|1000UN|UN.*|30UN.|30UN)$': 'UN',
        r'^(DZ|DÚZIA|DÚZIAS|DOZ|DZS|DZN|DZ12|DZ/1|DZ_|DZA|DZ?|DÚZ|DÃ?ZIA|DZ.*|DÃ?ZIA|DÃ{1}ZIA|DUZIAS)$': 'DZ',
        r'^(BDJ.*|BDJ|BJD|BANDEJ|BANDEJA|BANDEJAS|BAN|BAND|BNDJ|BJ?|BD30|BJA|BAD|BADJ|BANDJA|BNDJA|BDJAS|BJ1|BANJ|BANDE|BANEJ|BD30U)$': 'BANDEJA',
        r'^(PACOTE|PCT|PKG|PCTS?|PACOTES|PACTE|PCT100|PCT.5|PCT50|PC0001|PCT5|PKT)$': 'PACOTE',
        r'^(CART|CAR|CARTEL|CARTELA|CARTELAS|CA|CARTS|CT|CARTONADO|CT|CRT|CARTON?|CART.|CT30|CT/30|CTLA|TL|CRTELA|CATELA|CLT)$': 'CARTELA',
        r'^(FD|FARDO|FARDOS|FD20)$': 'FARDO',
        r'^(PL|PLACAS?|PLACA)$': 'PLACA',
        r'^(ALIM|ALIMENTAÇÃO|ALIMENTACOES|ALIMENTACAO)$': 'ALIMENTACAO',
        r'^(OUTROS|OUT|OTHER)$': 'OUTROS',
        r'^(LATA|LATAS|LAT125|LATA1|LT0001)$': 'LATA',
        r'^(FRD|FORMA|FORMAS|FRM)$': 'FORMA',
        r'^(MACO|MAÇOS?|MAC)$': 'MACO',
        r'^(KIT|JOGO)$': 'KIT',
        r'^(PENTE|PENTES|PN|PNT|PEN|PET|PENT)$': 'PENTE',
        r'^(FAVO|FAVOS)$': 'FAVO',
        r'^(TABELA|TBL|TB)$': 'TABELA',
        r'^(RL|ROL|ROLOS)$': 'ROLO',
        r'^(CUBA|CUBAS)$': 'CUBA',
        r'^(EMBAL|EMB|EMBL)$': 'EMBALAGEM',
        r'^(QT|QUILATOS?|QL|QUILAT)$': 'QUILATE',
        r'^(OVO|OVOS)$': 'UN',
        r'^(CONJUNTO|CJ)$': 'CONJUNTO',
        r'^(DISP|DISPOSITIVO)$': 'DISPOSITIVO',
        r'^(CRTL|CONTROLE|CR|CR.)$': 'CONTROLE',
        r'^(BANDEI|BANDEJA|BANDJA)$': 'BANDEJA',
        r'^(PT|POTE)$': 'POTE',
        r'^(SC|SACO|SACOS)$': 'SACO',
        r'^(DP|DISPENSADOR)$': 'DISPENSADOR',
        r'^(KM|KILOMETRO|KILOMETROS)$': 'KM'
    }

    for padrao, padronizado in padroes.items():
        if re.match(padrao, unidade):
            return padronizado

    return '-'


# Standardize units

In [6]:
# arruma e padroniza as unidades já existentes
df_dataset_beef['un_adj'] = df_dataset_beef['un'].apply(standardize_unit)

# ajustar unidade UN
# df_dataset_beef.loc[
#     (df_dataset_beef['un_adj'] == 'FARDO') &
#     (df_dataset_beef['item'].str.contains('unidade', case=False)),
#     'un_adj'
# ] = 'UN'

# ajustar unidade UN
df_dataset_beef.loc[
    (df_dataset_beef['un_adj'] == 'FARDO') &
    (df_dataset_beef['item'].str.contains('OVO DE GALINHA-FORMA', case=False)),
    'un_adj'
] = 'BANDEJA'


# Compute Stat

In [7]:
df_result = df_dataset_beef.groupby(['produtos', 'un_adj']).agg(
    count=('un_adj', 'count')
).reset_index()

# Adicionar a coluna de percentual
df_result['percent'] = df_result['count'] / df_result.groupby('produtos')['count'].transform('sum') * 100

# Opcional: arredondar o percentual para 2 casas decimais
df_result['percent'] = df_result['percent'].round(2)

df_result.query('produtos == "CARNE BOVINA"')

,produtos,un_adj,count,percent
28,CARNE BOVINA,-,140,0.01
29,CARNE BOVINA,500G,2,0.00
30,CARNE BOVINA,ALIMENTACAO,82,0.00
31,CARNE BOVINA,BANDEJA,2,0.00
32,CARNE BOVINA,CX,1159,0.05
33,CARNE BOVINA,DZ,144,0.01
34,CARNE BOVINA,EMBALAGEM,6,0.00
35,CARNE BOVINA,GRAMA,6626,0.29
36,CARNE BOVINA,KG,2232141,98.83
37,CARNE BOVINA,LATA,81,0.00


In [8]:
df_result.fillna(0).to_csv('data/stat_unidades_por_produto.csv')